**NOTE:**

Before running the codes, you need to download the main project folder to the main directory of your Google Drive.

# Libraries and Dataset

Import the necessary libraries and the TensorBoard notebook extension.

In [ ]:
%reset -f
import keras
import tensorflow as tf
from keras import layers
from keras.datasets import mnist
import numpy as np
from numpy import genfromtxt
from matplotlib import pyplot as plt
import matplotlib.image as mpimg 
from keras.callbacks import TensorBoard
from google.colab import files
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
import keras.backend as K
from IPython.display import clear_output 
%matplotlib inline
%reload_ext tensorboard

Load the MNIST data set

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Add random noise

In [ ]:
x_train_noisy = np.empty([len(x_train),28,28])

for i in range(len(x_train)):
  noise_factor = np.random.randint(0, 30)/100
  x_train_noisy[i] = x_train[i] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train[0].shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)

x_train = np.concatenate([x_train, x_train_noisy])
y_train = np.concatenate([y_train, y_train])

###Identity Block

In [ ]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
        
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)

    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):  
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters = F2, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def ResNet(input_shape, classes):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(32, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [32, 32], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [32, 32], stage=2, block='b')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [32, 32], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [32, 32], stage=3, block='b')
    
    # Stage 4
    X = convolutional_block(X, f = 3, filters = [32, 32], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [32, 32], stage=4, block='b')


    # Stage 5
    X = convolutional_block(X, f = 3, filters = [32, 32], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [32, 32], stage=5, block='b')
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    # X = AveragePooling2D(pool_size=(2,2), name='avg_pool')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    model.summary()

    return model

Training

In [ ]:
model = ResNet(input_shape = input_shape, classes = num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs = 2, batch_size = 32)

# Testing the ResNet classifier

We use 1000 MNIS test images to calculate the accuracy. First, load the trained model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model = load_model('/content/drive/MyDrive/KABAS-OGUTEN-EE3001-Term-Project/models/classification.h5') 

Calculate accuracy

In [ ]:
length = 1000
correct = 0
for i in range(length):
  img = x_test[i]
  img = np.reshape(img, (1, 28, 28, 1))
  pre = model.predict(img)
  if np.argmax(pre) == np.argmax(y_test[i]):
    correct = correct + 1
  clear_output()
  print(i, "/", length)

print("Accuracy: " + str(correct*100/length) + "%")

999 / 1000
Accuracy: 98.9%
